# Amazon DynamoDB 테이블과 인덱스

## DynamoDB Table
- Table : 다양한 아이템의 집합
- DynamoDB의 다양한 작업 단위(백업, 내보내기, 권한 등)
- 하나의 테이블에는 반드시 하나의 파티션키를 위한 속성 지정 필요
    - 선택적으로 정렬키를 위한 속성 지정도 가능
- 두가지 클래스
    - Standard : 표준
    - Standard-Infrequent Access : 저장은 저렴하지만 처리(읽기, 쓰기) 비용은 더 높음
- Capacity Mode
    - On-Demand : 읽고 쓴 용량만큼 과금
    - Provisioned Capacity : 시간단위당 릭고 쓸 수 있는 용량(WCU/RCU)을 미리 지정해서 과금

### DynamoDB Table Capacity Mode
- OnDemand : 읽고 쓴 용량만큼 과금
- Provisioned Capacity : 시간단위당 읽고 쓸 수 있는 용량(WCU/RCU)을 미리 지정해서 과금
    - OnDemand에 비해서 5배 이상 저렴
    - 필요 시 Reserved Capacity를 활용하여 추가 할인 가능
    - 초당 허용된 RCU/WCU 초과 시 Throttling 발생, 읽기 쓰기가 막힐 수 있음
    - Autoscaling 가능

#### DynamoDB Autoscaling
- Capacity를 상황에 따라 동적으로 조절 해주는 기능
- Target Utilization, 즉 수렴하고자 하는 사용량을 정해서 알고리즘에 따라 Capacity를 조절
    - 1분 단위로 측정하여 두번 이상 연속적으로 특정 사용량을 넘어선다면 Scale up : 2분 이상 소요
    - 1분 단위로 측정하여 15번 이상 연속적으로 특정 사용량보다 적다면 Scale Down : 15분 이상 소요
- 내부적으로 클라우드워치를 사용

### DynamoDB의 테이블 구조
- 각 DynamoDB 테이블은 PartitionKey의 Hash 값을 기준으로 각 10GB 단위 분산 저장
    - 각 파티션 단위로 3000RCU / 1000WCU를 넘을 경우 쓰로틀링 발생
- 즉 파티션 키는 최대한 유니크하게 유지할 필요가 있다

### DynamoDB의 인덱스
- 다이나모디비의 쿼리는 하나의 파티션 키만 활용 가능
- 테이블에 추가로 파티션 키 및 정렬 키를 부여하여 쿼리가 가능하도록 설정 가능
    - 일종의 View와 같은 개념
    - 모든 속성을 포함하거나 일부 속성만 포함하도록 설정 가능
- 두 가지 종류
    - Local Secondary Index
    - Global Secondary Index

#### Local Secondary Index
- 기존의 테이블 구조에서 정렬 키만 더 추가하는 인덱스
    - 파티션 내부에서 재정렬
    - RCU/WCU 등의 테이블 설정 공유
    - 정렬 키에 해당하는 값이 없다면 해당 항목은 제외
- 테이블 생성 시 같이 생성 필요
    - 테이블 생성 이후 추가 생성 불가능
- 최대 5개
- 새로 생성된 정렬 키와 파티션 키가 반드시 유니크일 필요 없음

#### Global Secondary Index
- 기존의 테이블 구조에서 새로운 파티션 키와 정렬 키 추가 가능
    - 추가로 파티션 생성 -> 일종의 서브 테이블을 따로 만드는 개념으로 스토리지 용량 추가
    - 별도의 RCU/WCU 등의 테이블 설정
- 언제든지 추가 가능
- 최종 일관성 읽기만 가능
    - 정확히는 GSI 자체가 업데이트를 할 때 최종 일관성 쓰기로 업데이트 진행
- 테이블 당 20개
- 새로 생성된 정렬 키와 파티션 키가 반드시 유니크일 필요 없음
- 선택적으로 포함할 속성들을 선택 가능
    - 쿼리할 때 해당 속성들의 크기를 합산해서 용량 계산에 반영

### Demo
1. 테이블 생성
    - demo-ddb-table / Partition Key : pk / Sort Key : sk
    - 로컬 인덱스 생성
        - Sort Key : sk2
        - 프로젝션 : All
2. 항목 생성
    - pk : pk1 / sk : sk1 /  sk2 : 11111 /  hype : movie / name : avengers
    - 항목 복제, 값들 바꾸자 (세번)
3. 쿼리 이리저리 해보자
4. 글로벌 보조 인덱스를 만들어보자